In [16]:
!cd ..

In [17]:
!pwd

/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata


In [18]:
from api_key import INTRINIO_USERNAME
from api_key import INTRINIO_PASSWORD             

import md_intrinio_client
import pandas as pd
import json

from md_intrinio_client import intrinio_get_company_metadata
from md_intrinio_client import intrinio_get_company_financials
from md_intrinio_client import intrinio_get_company_financials_csv
from md_intrinio_client import get_SandP_metadata
from md_intrinio_client import test_SandP_metadata


import sys
import json
import finsymbols
import ast
import requests

In [19]:
!pwd

/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata


In [20]:
import sys
#sys.path.append('/home/skillachie/Desktop/')
from finsymbols import symbols

In [21]:
import json 
SandP500 = {}
companyList = []
with open("SandP500_symbols.txt", "r") as fr:
            for line in fr:
                    company = json.loads(line)
                    SandP500[company["symbol"]] = line
                    companyList.append(company["symbol"])

tickerchunks = [companyList[x:x+100] for x in xrange(0, len(companyList), 100)]

In [22]:
print(len(tickerchunks[3]))
print((tickerchunks[3]))

100
[u'MMC', u'MLM', u'MAS', u'MA', u'MAT', u'MKC', u'MCD', u'MCK', u'MDT', u'MRK', u'MET', u'MTD', u'MGM', u'KORS', u'MCHP', u'MU', u'MSFT', u'MAA', u'MHK', u'TAP', u'MDLZ', u'MON', u'MNST', u'MCO', u'MS', u'MOS', u'MSI', u'MYL', u'NDAQ', u'NOV', u'NAVI', u'NTAP', u'NFLX', u'NWL', u'NFX', u'NEM', u'NWSA', u'NWS', u'NEE', u'NLSN', u'NKE', u'NI', u'NBL', u'JWN', u'NSC', u'NTRS', u'NOC', u'NRG', u'NUE', u'NVDA', u'ORLY', u'OXY', u'OMC', u'OKE', u'ORCL', u'PCAR', u'PKG', u'PH', u'PDCO', u'PAYX', u'PYPL', u'PNR', u'PBCT', u'PEP', u'PKI', u'PRGO', u'PFE', u'PCG', u'PM', u'PSX', u'PNW', u'PXD', u'PNC', u'RL', u'PPG', u'PPL', u'PX', u'PCLN', u'PFG', u'PG', u'PGR', u'PLD', u'PRU', u'PEG', u'PSA', u'PHM', u'PVH', u'QRVO', u'PWR', u'QCOM', u'DGX', u'Q', u'RRC', u'RJF', u'RTN', u'O', u'RHT', u'REG', u'REGN', u'RF']


In [23]:
# def generate_financial_data():
#     SandP500 = {}
#     companyList = []
#     with open("SandP500_symbols.txt", "r") as fr:
#             for line in fr:
#                     company = json.loads(line)
#                     SandP500[company["symbol"]] = line
#                     companyList.append(company["symbol"])

#     tickerchunks = [companyList[x:x+100] for x in xrange(0, len(companyList), 100)]

#     #print("Testing company financials api via CSV")

#     #years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
#     #quarters = ["Q1", "Q2", "Q3", "Q4", "FY"]
#     years = [2010]
#     quarters = ["Q1", "Q4"]
#     #companies = ["GE", "CSCO", "GOOG", "FACE"]

#     bigDict = {}

#     loopindex = 0
#     for company in tickerchunks[2]:
#         print("working on {}".format(company))
#         if loopindex > 2:
#             break
#         loopindex += 1
#         bigDict[company] = {}
#         #print("working on company {}".format(company))
#         for year in years:
#             bigDict[company][year] = {}
#             print("working on year {}".format(year))
#             for quarter in quarters:
#                 print("working on quarter {}".format(quarter))
#                 bigDict[company][year][quarter] = {}
#                 data = intrinio_get_company_financials_csv(company, str(year), quarter)
#                 print(data)
#                 financials = list(data)

#                 print("FINANCIALS")
#                 print (financials)
#                 breakdown = ''.join(financials)
#                 #print(breakdown)
#                 newdata = breakdown.split("\n")
#                 print(newdata)
#                 for item in newdata:
#                     splititem = item.split(",")
#                     if len(splititem) < 2:
#                         continue
#                     bigDict[company][year][quarter][splititem[0]]=  {}
#                     bigDict[company][year][quarter][splititem[0]]= splititem[1]
#         print("DONE with {}".format(company))      
        
        
#         print("Done with {}".format(company))
#         with open("./data/"+company+"_Financials_by_Quarter.json", 'w') as fp:
#             fp.write(json.dumps(bigDict))
#             print("{}".format("./data/"+company+"_Financials_by_Quarter.json"))
#             bigDict = {}
#         return

In [24]:
def new_intrinio_get_company_financials(symbol, year, quarter):
        # Get the latest FY Income Statement for "symbol"
        # 'type': 'FY'
        cleanedupdata = {}
        base_url = "https://api.intrinio.com"
        request_url = base_url + "/financials/standardized"
        query_params = {
                'ticker': symbol,
                'statement': 'income_statement',
                'fiscal_year' : str(year),
                'fiscal_period' : quarter
        }

        response = requests.get(request_url, params=query_params, auth=(INTRINIO_USERNAME, INTRINIO_PASSWORD))
        if response.status_code == 401: print("Unauthorized! Check your username and password."); exit()

        if response.status_code == 429:
            print("API query limit reached")
            return
        data = response.json()['data']

        #print(data['basicdilutedeps'])
        for row in data:
                tag = row['tag']
                value = row['value']
                cleanedupdata["AASYMBOL"] = symbol
                cleanedupdata["ABYEAR"] = year
                cleanedupdata["ACPeriod"] = quarter
                cleanedupdata[tag] = value
                #print(tag + ": " + str(value))
#         for k, v in xx.items():
#             print(k, v)
        datalist=[]
        for key, value in sorted(cleanedupdata.items()):
            datalist.append(str(value))
        
        #print(json.dumps(cleanedupdata, indent=1, sort_keys=True))
        #print(datalist)
        #print("{}".format("".join(datalist)))
        #return(",".join(datalist))
        return(datalist)

In [25]:
def updated_cleanupdata(cleanedupdata):
    newData = {}
    newData['AASYMBOL'] = cleanedupdata['AASYMBOL']
    newData['ABYEAR'] = cleanedupdata['ABYEAR']
    newData['ACPeriod'] = cleanedupdata['ACPeriod']
    newData['basicdilutedeps'] = cleanedupdata.get('basicdilutedeps', 0.0)
    newData['basiceps'] = cleanedupdata.get('basiceps', 0.0)
    
    newData['cashdividendspershare'] = cleanedupdata.get('cashdividendspershare', 0.0)
    newData['dilutedeps'] = cleanedupdata.get('dilutedeps', 0.0)
    newData['incometaxexpense'] = cleanedupdata.get('incometaxexpense', 0.0)
    newData['netincome'] = cleanedupdata.get('netincome', 0.0)
    newData['netincomecontinuing'] = cleanedupdata.get('netincomecontinuing', 0.0)
    
    newData['netincomediscontinued'] = cleanedupdata.get('netincomediscontinued', 0.0)
    newData['netincometocommon'] = cleanedupdata.get('netincometocommon', 0.0)
    newData['netincometononcontrollinginterest'] = cleanedupdata.get('netincometononcontrollinginterest', 0.0)
    newData['operatingcostofrevenue'] = cleanedupdata.get('operatingcostofrevenue', 0.0)
    newData['operatingrevenue'] = cleanedupdata.get('operatingrevenue', 0.0)
    
    newData['othercostofrevenue'] = cleanedupdata.get('othercostofrevenue', 0.0)
    newData['otherincome'] = cleanedupdata.get('otherincome', 0.0)
    newData['preferreddividends'] = cleanedupdata.get('preferreddividends', 0.0)
    newData['sgaexpense'] = cleanedupdata.get('sgaexpense', 0.0)
    newData['totalcostofrevenue'] = cleanedupdata.get('totalcostofrevenue', 0.0)
    
    newData['totalgrossprofit'] = cleanedupdata.get('totalgrossprofit', 0.0)
    newData['totalinterestexpense'] = cleanedupdata.get('totalinterestexpense', 0.0)
    newData['totaloperatingexpenses'] = cleanedupdata.get('totaloperatingexpenses', 0.0)
    newData['totaloperatingincome'] = cleanedupdata.get('totaloperatingincome', 0.0)
    newData['totalotherincome'] = cleanedupdata.get('totalotherincome', 0.0)
    
    newData['totalpretaxincome'] = cleanedupdata.get('totalpretaxincome', 0.0)
    newData['totalrevenue'] = cleanedupdata.get('totalrevenue', 0.0)
    newData['weightedavebasicdilutedsharesos'] = cleanedupdata.get('weightedavebasicdilutedsharesos', 0.0)
    newData['weightedavebasicsharesos'] = cleanedupdata.get('weightedavebasicsharesos', 0.0)
    newData['weightedavedilutedsharesos'] = cleanedupdata.get('weightedavedilutedsharesos', 0.0)
    
    #newData[] = cleanedupdata.get(, 0.0)
    return newData

In [26]:
#attributes = ["basiceps", "netincomediscontinued", "weightedavedilutedsharesos", "incometaxexpense", "netincometocommon", "cashdividendspershare", "totaloperatingincome", "weightedavebasicsharesos", "basicdilutedeps", "operatingcostofrevenue", "totalotherincome", "totalrevenue", "dilutedeps", "otherincome", "TAG", "totalgrossprofit", "rdexpense", "totalpretaxincome", "totaloperatingexpenses", "totalcostofrevenue", "weightedavebasicdilutedsharesos", "RESULT_COUNT: 26", "sgaexpense", "operatingrevenue", "totalinterestincome", "netincomecontinuing", "netincome", "totalinterestexpense"]
#print(len(attributes))

# attributes = ["ticker", "year", "quarter", "basicdilutedeps", "basiceps", "dilutedeps", "incometaxexpense", 
#               "netincome","netincomecontinuing", "netincometocommon", "operatingcostofrevenue","operatingrevenue",
#               "otherincome", "rdexpense", "sgaexpense", "totalcostofrevenue", "totalgrossprofit", "totalinterestexpense",
#               "totalinterestincome", "totaloperatingexpenses", "totaloperatingincome", "totalotherincome", 
#               "totalpretaxincome", "totalrevenue", "weightedavebasicdilutedsharesos","weightedavebasicsharesos", 
#               "weightedavedilutedsharesos"]
attributes = ["ticker", "year", "quarter", "basicdilutedeps", "basiceps",
              'cashdividendspershare', 'dilutedeps', 'incometaxexpense', 'netincome', 'netincomecontinuing',
              'netincomediscontinued', 'netincometocommon', 'netincometononcontrollinginterest', 'totalcostofrevenue', 
                  'operatingcostofrevenue', 'operatingrevenue',
              'othercostofrevenue', 'otherincome', 'preferreddividends', 'sgaexpense', 
              'totalgrossprofit', 'totalinterestexpense', 'totaloperatingexpenses', 'totaloperatingincome', 'totalotherincome', 
              'totalpretaxincome', 'totalrevenue', 'weightedavebasicdilutedsharesos', 'weightedavebasicsharesos', 'weightedavedilutedsharesos'
                 ]
print(len(attributes))

xx = ",".join(attributes)
print(xx)

30
ticker,year,quarter,basicdilutedeps,basiceps,cashdividendspershare,dilutedeps,incometaxexpense,netincome,netincomecontinuing,netincomediscontinued,netincometocommon,netincometononcontrollinginterest,totalcostofrevenue,operatingcostofrevenue,operatingrevenue,othercostofrevenue,otherincome,preferreddividends,sgaexpense,totalgrossprofit,totalinterestexpense,totaloperatingexpenses,totaloperatingincome,totalotherincome,totalpretaxincome,totalrevenue,weightedavebasicdilutedsharesos,weightedavebasicsharesos,weightedavedilutedsharesos


In [27]:
def updated_intrinio_get_company_financials(symbol, year, quarter):
        # Get the latest FY Income Statement for "symbol"
        # 'type': 'FY'
        cleanedupdata = {}
        base_url = "https://api.intrinio.com"
        request_url = base_url + "/financials/standardized"
        query_params = {
                'ticker': symbol,
                'statement': 'income_statement',
                'fiscal_year' : str(year),
                'fiscal_period' : quarter
        }

        response = requests.get(request_url, params=query_params, auth=(INTRINIO_USERNAME, INTRINIO_PASSWORD))
        if response.status_code == 401: print("Unauthorized! Check your username and password."); exit()

        if response.status_code == 429:
            print("API query limit reached")
            return
        data = response.json()['data']

        #print(data['basicdilutedeps'])
        for row in data:
                #print(row)
                tag = row['tag']
                value = row['value']

                cleanedupdata[tag] = value
                #print(tag + ": " + str(value))
#         for k, v in xx.items():
#             print(k, v)
        datalist=[]
        attr = []
        cleanedupdata["AASYMBOL"] = symbol
        cleanedupdata["ABYEAR"] = year
        cleanedupdata["ACPeriod"] = quarter
        cleanedupdata = updated_cleanupdata(cleanedupdata)
        for key, value in sorted(cleanedupdata.items()):
            datalist.append(str(value))
            attr.append(str(key))
        #return (json.dumps(cleanedupdata, indent=1, sort_keys=True))
        #print(datalist)
        #print("{}".format("".join(datalist)))
        #return(",".join(datalist))
        return(datalist, attr)
#data = intrinio_get_company_financials('GE', '2010', 'FY')

In [35]:
def generate_financial_data():
    SandP500 = {}
    companyList = []
    with open("SandP500_symbols.txt", "r") as fr:
            for line in fr:
                    company = json.loads(line)
                    SandP500[company["symbol"]] = line
                    companyList.append(company["symbol"])

    tickerchunks = [companyList[x:x+100] for x in xrange(0, len(companyList), 100)]

    #print("Testing company financials api via CSV")

    years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
    quarters = ["Q1", "Q2", "Q3", "Q4", "FY"]
    #years = [2010]
    #quarters = ["Q1", "Q2"]
    #companies = ["GE", "CSCO", "GOOG", "FACE"]

    bigDict = {}

    attributes = ["ticker", "year", "quarter", "basicdilutedeps", "basiceps",
              'cashdividendspershare', 'dilutedeps', 'incometaxexpense', 'netincome', 'netincomecontinuing',
              'netincomediscontinued', 'netincometocommon', 'netincometononcontrollinginterest', 'totalcostofrevenue', 
                  'operatingcostofrevenue', 'operatingrevenue',
              'othercostofrevenue', 'otherincome', 'preferreddividends', 'sgaexpense', 
              'totalgrossprofit', 'totalinterestexpense', 'totaloperatingexpenses', 'totaloperatingincome', 'totalotherincome', 
              'totalpretaxincome', 'totalrevenue', 'weightedavebasicdilutedsharesos', 'weightedavebasicsharesos', 'weightedavedilutedsharesos'
                 ]
    print(len(attributes))

    xx = ",".join(attributes) + "\n"

    # tickerchunks[2] needs to be re-done with header line
    for company in tickerchunks[3]:
        with open("./revenue/"+company+"_Financials_by_Quarter.csv", 'a') as fw:
            fw.write(xx)
            print("working on {}".format(company))
#             if loopindex > 2:
#                 break
#             loopindex += 1
            #print("working on company {}".format(company))
            for year in years:
                #bigDict[company][year] = {}
                #print("working on year {}".format(year))
                for quarter in quarters:
                    #print("working on quarter {}".format(quarter))
                    #bigDict[company][year][quarter] = {}
                    data, _ = updated_intrinio_get_company_financials(company, str(year), quarter)
                    yy = ",".join(data)
                    yy +="\n"
                    #print(data)
                    #print(",".join(data))
                    fw.write(yy)

            print("DONE with {}".format(company))      

In [36]:
generate_financial_data()

30
working on MMC
DONE with MMC
working on MLM
DONE with MLM
working on MAS
DONE with MAS
working on MA
DONE with MA
working on MAT
DONE with MAT
working on MKC
DONE with MKC
working on MCD
DONE with MCD
working on MCK
DONE with MCK
working on MDT
DONE with MDT
working on MRK
DONE with MRK
working on MET
DONE with MET
working on MTD
DONE with MTD
working on MGM
DONE with MGM
working on KORS
DONE with KORS
working on MCHP
DONE with MCHP
working on MU
DONE with MU
working on MSFT
DONE with MSFT
working on MAA
DONE with MAA
working on MHK
DONE with MHK
working on TAP
DONE with TAP
working on MDLZ
DONE with MDLZ
working on MON
DONE with MON
working on MNST
DONE with MNST
working on MCO
DONE with MCO
working on MS
DONE with MS
working on MOS
DONE with MOS
working on MSI
DONE with MSI
working on MYL
DONE with MYL
working on NDAQ
DONE with NDAQ
working on NOV
DONE with NOV
working on NAVI
DONE with NAVI
working on NTAP
DONE with NTAP
working on NFLX
DONE with NFLX
working on NWL
DONE with NWL

In [12]:
attributes = ["basiceps", "netincomediscontinued", "weightedavedilutedsharesos", "incometaxexpense", "netincometocommon", "cashdividendspershare", "totaloperatingincome", "weightedavebasicsharesos", "basicdilutedeps", "operatingcostofrevenue", "totalotherincome", "totalrevenue", "dilutedeps", "otherincome", "TAG", "totalgrossprofit", "rdexpense", "totalpretaxincome", "totaloperatingexpenses", "totalcostofrevenue", "weightedavebasicdilutedsharesos", "RESULT_COUNT: 26", "sgaexpense", "operatingrevenue", "totalinterestincome", "netincomecontinuing", "netincome", "totalinterestexpense"]
print(len(attributes))

attributes = ["ticker", "year", "quarter", "basicdilutedeps", "basiceps", "dilutedeps", "incometaxexpense", 
              "netincome","netincomecontinuing", "netincometocommon", "operatingcostofrevenue","operatingrevenue",
              "otherincome", "rdexpense", "sgaexpense", "totalcostofrevenue", "totalgrossprofit", "totalinterestexpense",
              "totalinterestincome", "totaloperatingexpenses", "totaloperatingincome", "totalotherincome", 
              "totalpretaxincome", "totalrevenue", "weightedavebasicdilutedsharesos","weightedavebasicsharesos", 
              "weightedavedilutedsharesos"]
print(len(attributes))

28
27


In [8]:
xx = ",".join(attributes)
print(xx)

NameError: name 'attributes' is not defined

In [56]:
import os
os.chdir("/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata")
!pwd

/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata


In [52]:
import glob, os
import ast

def convert_json_to_csv():
    companyDict = {}
    os.chdir("./data")
    #print("got here")
    for datafile in glob.glob("*Financials*.json"):
        #print(datafile)
        with open(datafile, "r") as fr:
            #print ("Processing {}".format(datafile))
            for line in fr:
                companyDict = ast.literal_eval(line)
                
                # Save company ticker
                for company, companyvalue in companyDict.items():
                    #print("Company is")
                    #print("{}".format(company, end=''))
                    # For every year subset
                    if isinstance(companyvalue, dict):
                        for year, yearvalue in companyvalue.items():
                            #print("Year is")
                            #print("{}".format(year, end=''))
                            # Look at the corresponding quarter
                            if isinstance(yearvalue, dict):
                                datalist=[]
                                # Look at company quarter
                                for quarter, quartervalue in yearvalue.items():
                                    #print("Quarter is")
                                    #print("{}".format(quarter, end=''))
                                    for financialdata in quartervalue.values():
                                        datalist.append(financialdata)
                                    
                                    if len(datalist) > 21:
                                        datalist[14] = "IGNORE"
                                        datalist[21] = "IGNORE"
                                    
                                    if len(datalist) == 2:
                                        datalist[0] = ""
                                        datalist[1] = ""
                                    print("{},{},{},{}".format(company, year, quarter, ",".join(datalist), end=''))
                                    
                                    datalist = []
        break
    
    

In [53]:
convert_json_to_csv()

FLIR,2010,Q1,
FLIR,2010,Q4,


In [ ]:
def get_company_financial_data_all(symbol):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        #companyDict = json.load(fp)
        for line in fp:
            companyDict = ast.literal_eval(line)

        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict[symbol]
            

In [ ]:
get_company_financial_data_all("GE")

In [ ]:
def get_company_financial_data_by_year(symbol, year):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        #companyDict = json.load(fp)
        for line in fp:
            companyDict = ast.literal_eval(line)

        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict[symbol][year]
            

In [ ]:
get_company_financial_data_by_year("GE", "2014")

In [ ]:
def get_company_financial_data_by_year_and_attr(symbol, year, attribute):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        #companyDict = json.load(fp)
        for line in fp:
            companyDict = ast.literal_eval(line)

        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict[symbol][year][attribute]
            

In [ ]:
get_company_financial_data_by_year_and_attr("GE", "2014", "basiceps")

In [ ]:
def generate_company_financial_data_csv(symbol):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        for line in fp:
            companyDict = ast.literal_eval(line)
            
        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict
            
            
            